# 30x30 at the state level: Implementation and progress 

In [ ]:
from cng.utils import *
from cng.h3 import *
from ibis import _
import ibis.selectors as s
import geopandas as gpd
import pandas as pd
from sources import *
from mapping_utils import *

con = ibis.duckdb.connect(extensions = ["spatial"])
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
duckdb_install_h3()

set_secrets(con)

# Loading data, creating one dataframe
- `sources.py` contains dicts with sources
- I added polygons to each state

In [ ]:
# imported from sources.py
impl_df = pd.DataFrame(implementation_sources)
prog_df = pd.DataFrame(progress_sources)

# joining into single dataframe 
df = impl_df.merge(
    prog_df,
    on="state_id",
    how="outer",
    validate="m:m"
)

# adding polygon boundaries for states using Census data
census_url = 's3://public-census/2024/state/2024_us_state.parquet'
polys = con.read_parquet(census_url).select('state_id','geom').filter(~_.state_id.isin(['AS','DC','GU','MP','PR','VI'])).execute()

gdf = df.merge(polys, on="state_id", how="outer").set_geometry('geom').set_crs("EPSG:4326")

# Mapping implementation type 

In [ ]:
impl_gdf = gdf.drop(columns=['progress_type','progress_notes','progress_citation','progress_url'])

# splitting up geometries for states that have multiple entries to map as a checkboard pattern 
impl_gdf = checkerboard_geom(impl_gdf,type_col="implementation_type",tile_m=80_000)

In [ ]:
import leafmap.maplibregl as leafmap
m = leafmap.Map(style="positron")
           
eo_color = "#918EF4"
ec_color = "#42BFDD"
law_color = "#568046"
other_color = "#090088"
resolution_color = "#F28C28"
proposed_color = "#CC0000"

paint = {
    "fill-color": [
        "match",
        ["get", "implementation_type"],     
        "Executive Order", eo_color,    
        "Executive Commitment", ec_color,
        "Statute", law_color,
        "Proposed Legislation", proposed_color,
        "Legislative Resolution", resolution_color,
        "Statute (Other)", other_color,    
        "#FFFFFF"  # default for none/anything else
    ],
    "fill-opacity": 0.9
}
m.add_gdf(impl_gdf, paint = paint, layer_type = 'fill')

legend_dict = {
    "Statute": law_color,
    "Statute (Other)": other_color,
    "Executive Order": eo_color, 
    "Executive Commitment": ec_color,
    "Legislative Resolution": resolution_color, 
    "Proposed Bill": proposed_color,
}

m.add_legend(
    title="",
    legend_dict=legend_dict,
    # position='top-right'
)

# Add a boundary line layer
m.add_gdf(
    impl_gdf,
    layer_type="line",
    paint={
        "line-color": "#000000",   # black borders
        "line-width": 0.8
    },
    name="borders",
    fit_bounds=True)


m

# Mapping progress type 

In [ ]:
prog_gdf = gdf.drop(columns=['implementation_type','implementation_notes','implementation_citation','implementation_url'])

# splitting up geometries for states that have multiple entries to map as a checkboard pattern 
prog_gdf = checkerboard_geom(prog_gdf, type_col="progress_type",tile_m=80_000)

In [ ]:
import leafmap.maplibregl as leafmap
m = leafmap.Map(style="positron")

paint = {
    "fill-color": [
        "match",
        ["get", "progress_type"],     
         "Achieved", "#568046",
        "Active", "#7DC7FF",
        "Halted", "#E05241",
        "Failed Legislation", "#F28C28",
        "#FFFFFF"  # default for none/anything else
    ],
    "fill-opacity": 0.9
}
m.add_gdf(prog_gdf, paint = paint, layer_type = 'fill')

legend_dict = {
    "Achieved": "#568046",
    "Active": "#7DC7FF",
    "Halted": "#E05241",
    "Failed Legislation": "#F28C28",
}


m.add_legend(
    title="",
    legend_dict=legend_dict,
    # position='top-right'
)

# Add a boundary line layer
m.add_gdf(
    prog_gdf,
    layer_type="line",
    paint={
        "line-color": "#000000",   # black borders
        "line-width": 0.8
    },
    name="borders"
)

m